# BAN5753 EXERCISE 2

## Part I

1.	Build a regression model to predict Satisf using only dealers’ perceived importance about choosing a supplier of hydraulic and pneumatic products. Variable names in the data are Reliab2, Time2, ….,Return2, Warranty2 as independent variables.

In [ ]:
# import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
# import necessary package to handle excel workbook
import openpyxl
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.feature_selection import RFE, RFECV
import numpy as np

In [ ]:
# import data
df = pd.read_excel("C:\\Users\\cwolt\\OneDrive\\Desktop\\BAnDS\\Fall 2025\\Advanced Business Analytics\\Module 2\\BFP1_Data.xlsx")
df.head()

In [ ]:
# Show column headers
df.columns

In [ ]:
# transpose the outpute of the df
df.describe().T

In [ ]:
model_df = df[['reliab2', 'time2', 'av_br2', 'av_spec2', 'price2', 'credit2', 'return2', 'warrant2']]
# check for null values in subsetted data
model_df.isnull().sum()

In [ ]:
# drop all null values
model_df = model_df.dropna(axis=0, how='any')

In [ ]:
# checking the data types of the columns to ensure they can be used for model building
model_df.dtypes

In [ ]:
# define the model formula
ols_formula = """
satisf ~ 
reliab2 + 
time2 + 
av_br2 + 
av_spec2 + 
price2 + 
credit2 + 
return2 + 
warrant2
"""
# fit the regression model and print the model diagnostics
linear_model = smf.ols(ols_formula, df).fit()
linear_model.summary()

In [ ]:
# to calculate RMSE and CV
# residuals and mean of Y
residuals = linear_model.resid
mean_y = 8.598706 # from above descriptive statistics

# Root Mean Square Error (RMSE)
rmse = np.sqrt(np.mean(residuals**2))

# Coefficient of Variation (CV)
cv = (rmse / mean_y) * 100

print(f"RMSE: {rmse:.3f}")
print(f"CV: {cv:.2f}%")


## Part II
2.	Create a new variable (name it as Avg_Score) by averaging the three variables, Satisf, Rate and Percent. Then, run a stepwise regression with Avg_Score as dependent variable and Time2,……, Return2, Warrant2 as well as I_access, Puch_I, pr_area, num_emp and Industry as independent variables. Use p-value to enter as 0.05 and p-value to stay as 0.05.

In [ ]:
# create new variable for average_score
Avg_Score = np.mean(df[['satisf', 'rate', 'percent']], axis=1)
print(Avg_Score)

In [ ]:
# create new df for stepwise regression model
model_df2 = df[['time2', 'av_br2', 'av_spec2', 'price2', 'credit2', 'return2', 'warrant2', 'i_acces', 'purch_i', 'pr_area', 'num_emp', 'industry']]
# add new variable into df
model_df2['Avg_Score'] = Avg_Score

# check for all null values
model_df2.isnull().sum
model_df2.dropna(axis=0, how = 'any')


In [ ]:
model_df2.info()

In [ ]:
# subset the columns easily from the data frame
def _model_data_subset(input_data: pd.DataFrame, column_subset: list =None) -> pd.DataFrame:
    if column_subset is None:
        return input_data
    else:
        _model_subset_data = input_data[column_subset]
        return _model_subset_data

In [ ]:
# create stepwise regression model
def _stepwise_selection_p_val(
        input_dataframe: pd.DataFrame,
        target_variable_name: str,
        column_subset: list = None,
        SL_in: float = 0.05,
        SL_out: float = 0.05,
) -> [pd.DataFrame, pd.Series, list]:
    model_subset_data = _model_data_subset(input_dataframe, column_subset)
    model_subset_data_nona = model_subset_data.dropna()

    target = model_subset_data_nona[target_variable_name]

    x_variables_subset_data = model_subset_data_nona.drop(target_variable_name, axis = 1)

    initial_features = x_variables_subset_data.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(
                target,
                sm.add_constant(model_subset_data_nona[best_features + [new_column]]),
            ).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < SL_in:
            best_features.append(new_pval.idxmin())
            while len(best_features) > 0:
                best_features_with_constant = sm.add_constant(
                    model_subset_data_nona[best_features]
                )
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if max_p_value >= SL_out:
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break
        else:
            break
    return x_variables_subset_data, target, best_features

In [ ]:
def build_linear_regression_stepwise_based(
        input_dataframe: pd.DataFrame,
        target_variable_name: str,
        column_subset: list = None,
        SL_in: float = 0.05,
        SL_out: float = 0.05,
) -> [list, list]:
    x_variable_subset_data, target_variable, best_features = _stepwise_selection_p_val(
        input_dataframe, target_variable_name, column_subset, SL_in=SL_in, SL_out=SL_out
    )
    dataframe_subset_best_features = x_variable_subset_data[best_features]
    regression_model = sm.OLS(
        target_variable, sm.add_constant(dataframe_subset_best_features)
    ).fit()
    print("The best features are ", best_features)
    return regression_model.summary()


In [ ]:
SL_in = 0.05 # entry p-value
SL_out = 0.05 # exit p-value

build_linear_regression_stepwise_based(model_df2, "Avg_Score",['time2', 
                                                               'av_br2', 
                                                               'av_spec2', 
                                                               'price2', 
                                                               'credit2', 
                                                               'return2', 
                                                               'warrant2', 
                                                               'i_acces', 
                                                               'purch_i', 
                                                               'pr_area', 
                                                               'num_emp', 
                                                               'industry'], 0.05, 0.05)